In [1]:
import boost_histogram as bh
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl


# =========================
# LOAD RUN METADATA
# =========================

runs_df = pd.read_csv("/Volumes/T7/ROOTfiles/LH2/-3p632gev7p87deg/LH2.csv")

runs_df["weight_factor"] = 1 / (runs_df["charge"] * runs_df["hEff"] * runs_df["hDead"])

Al_thick = 0.219  # mm
dummy_thick = 0.60605  # mm
R = Al_thick / dummy_thick
normfac = 0.838913E+11 


# =========================
# HELPER FUNCTIONS
# =========================

def load_branch(filename, branches, run_type):
    with uproot.open(filename) as file:
        if run_type == "sim":
            tree = file["h10"]  
        else:
            tree = file["T"]     
        data = {br: tree[br].array(library="np") for br in branches}
    return pd.DataFrame(data)


def data_cuts(df):
    mask = (
        (df["H.gtr.dp"] > -8) & (df["H.gtr.dp"] < 8) &
        (df["H.cal.etottracknorm"] > 0.8) &
        (df["P.gtr.dp"] > -10) & (df["P.gtr.dp"] < 22) &
        (df["P.ngcer.npeSum"] < 1) &
        (df["P.cal.etottracknorm"] < 0.4) &
        (df["CTime.ePiCoinTime_ROC1"]>50) & 
        (df["CTime.ePiCoinTime_ROC1"]<52)
    )
    return df[mask]

def random_cuts(df):
    mask = (
        (df["H.gtr.dp"] > -8) & (df["H.gtr.dp"] < 8) &
        (df["H.cal.etottracknorm"] > 0.8) &
        (df["P.gtr.dp"] > -10) & (df["P.gtr.dp"] < 22) &
        (df["P.ngcer.npeSum"] < 1) & 
        (df["P.cal.etottracknorm"] < 0.4) &
        (df["CTime.ePiCoinTime_ROC1"]>26) & 
        (df["CTime.ePiCoinTime_ROC1"]<46)
    )
    return df[mask]


def sim_cuts(df):
    """Apply physics cuts to DataFrame for SIMULATION."""
    mask = (
        (df["hsdelta"] > -8) & (df["hsdelta"] < 8) &
        (df["ssdelta"] > -10) & (df["ssdelta"] < 22)
    )
    return df[mask]


def fill_histogram(data, weights, bins, range):
    """Fill a boost-histogram and return counts and errors."""
    hist = bh.Histogram(
        bh.axis.Regular(bins, range[0],range[1]),
        storage=bh.storage.Weight()
        )
    hist.fill(data, weight=weights)
    hist_view = hist.view()
    counts = hist_view['value']
    errors = np.sqrt(hist_view['variance'])
    bin_centers = hist.axes[0].centers
    
    return bin_centers, counts, errors

# =========================
# LOAD AND PROCESS DATA
# =========================

def process_runs(run_type):
    """Load all runs of a given type, apply cuts, return concatenated variable & weights."""
    subset = runs_df[runs_df["run_type"] == run_type]
    all_data = []
    all_weights = []

    if run_type in ["data", "dummy"]:
        branches = [
            "H.gtr.dp", "H.cal.etottracknorm", "H.cer.npeSum",
            "P.gtr.dp", "P.cal.etottracknorm", "P.ngcer.npeSum", "P.hgcer.npeSum",
            "CTime.ePiCoinTime_ROC1", "H.gtr.y", "P.gtr.y", "H.kin.primary.x_bj",
            "H.kin.primary.Q2", "P.gtr.p", "H.kin.primary.nu"
        ]
        for _, row in subset.iterrows():
            df = load_branch(row.filename, branches, row.run_type)
            df_cut = data_cuts(df)
            all_data.append(df_cut[var_to_plot[0]].to_numpy())
            all_weights.append(np.full(len(df_cut), row.weight_factor))

    elif run_type == "sim":
        branches = [
            "hsdelta", "ssdelta", "Weight", "hsytar", "ssytar", "xbj", "nu", "Q2", "z"
        ]
        for _, row in subset.iterrows():
            df = load_branch(row.filename, branches, row.run_type)
            df_cut = sim_cuts(df)
            # Normalize sim weights
            w = df_cut["Weight"].to_numpy() * normfac / len(df_cut)
            all_data.append(df_cut[var_to_plot[1]].to_numpy())
            all_weights.append(w)      
    return np.concatenate(all_data), np.concatenate(all_weights)

def process_random(run_type):
    """Load all runs of a given type, apply cuts, return concatenated variable & weights."""
    subset = runs_df[runs_df["run_type"] == run_type]
    all_data = []
    all_weights = []

    if run_type in ["data", "dummy"]:
        branches = [
            "H.gtr.dp", "H.cal.etottracknorm", "H.cer.npeSum",
            "P.gtr.dp", "P.cal.etottracknorm", "P.ngcer.npeSum", "P.hgcer.npeSum",
            "CTime.ePiCoinTime_ROC1", "H.gtr.y", "P.gtr.y", "H.kin.primary.x_bj",
            "H.kin.primary.Q2", "P.gtr.p", "H.kin.primary.nu"
        ]
        for _, row in subset.iterrows():
            df = load_branch(row.filename, branches, row.run_type)
            df_cut = random_cuts(df)
            all_data.append(df_cut[var_to_plot[0]].to_numpy())
            all_weights.append(np.full(len(df_cut), row.weight_factor))      
    return np.concatenate(all_data), np.concatenate(all_weights)


def make_histograms(bins, range):

    # Process all runs
    data_vals, data_weights = process_runs("data")
    dummy_vals, dummy_weights = process_runs("dummy")
    random_vals, random_weights = process_random("data")
    sim_vals, sim_weights = process_runs("sim")

    # Fill histograms
    centers_data, counts_data, errors_data = fill_histogram(data_vals, data_weights, bins, range)
    centers_dummy, counts_dummy, errors_dummy = fill_histogram(dummy_vals, dummy_weights, bins, range)
    centers_random, counts_random, errors_random = fill_histogram(random_vals, random_weights, bins, range)
    centers_sim, counts_sim, errors_sim = fill_histogram(sim_vals, sim_weights, bins, range)

    # Vectorized dummy subtraction
    counts_sub = counts_data - R * counts_dummy - (1/5) * counts_random
    errors_sub = np.sqrt(errors_data**2 + (R * errors_dummy)**2 + ((1/5) * errors_random)**2)
    centers_sub = centers_data

    return {
        "centers_data": centers_data,
        "counts_data": counts_data,
        "errors_data": errors_data,
        "centers_dummy": centers_dummy,
        "counts_dummy": counts_dummy,
        "errors_dummy": errors_dummy,
        "centers_sim": centers_sim,
        "counts_sim": counts_sim,
        "errors_sim": errors_sim,
        "centers_sub": centers_sub,
        "counts_sub": counts_sub,
        "errors_sub": errors_sub,
        "centers_random": centers_random,
        "counts_random": counts_random,
        "errors_random": errors_random
    }


# HMS Delta

In [2]:
import mplhep as hep
# plt.style.use(hep.style.ROOT)
plt.style.use("fivethirtyeight")

bin_num = 32
var_to_plot = ["H.gtr.dp","hsdelta"]
bin_range = (-8, 8)
Al_thick = 0.184  # mm
dummy_thick = 0.60605  # mm
R = Al_thick / dummy_thick
normfac = 0.838913E+11 
histo = make_histograms(bin_num, bin_range)

diff_percent = 100 * (histo["counts_sub"] - histo["counts_sim"]) / histo["counts_sim"]
mean_diff = np.mean(np.abs(diff_percent))
print(f"Average relative difference: {mean_diff:.2f}%")

plt.figure(figsize=(20, 12))

# plt.step(histo["centers_data"], histo["counts_data"], where='mid', label='Data', color='blue')
# plt.step(histo["centers_sim"], histo["counts_sim"], where='mid', label='SIMC', color='black')
# plt.fill_between(histo["centers_sub"], 0, histo["counts_sub"], step='mid', alpha=0.3, color='red', label='Dummy-subtracted')
plt.errorbar(histo["centers_data"], histo["counts_data"], yerr=histo["errors_data"], fmt='o', color='blue', label='LH2 Data(combined runs)')
plt.errorbar(histo["centers_sim"], histo["counts_sim"], yerr=histo["errors_sim"], fmt='s', color='black', label='SIMC')
plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='red', label="Dummy/random-subtracted data")
plt.xlabel("H.gtr.dp [%]",fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(1, max(histo["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title("HMS delta")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/juliogutierrez/Downloads/R-sidis_analysis/ROOTfiles/LH2/-3p632gev7p87deg/coin_replay_production_24091_-1.root'

# SHMS Delta

In [ ]:
plt.style.use("fivethirtyeight")
bin_num = 64
var_to_plot = ["P.gtr.dp","ssdelta"]
bin_range = (-10, 22)

histo = make_histograms(bin_num, bin_range)

diff_percent = 100 * (histo["counts_sub"] - histo["counts_sim"]) / histo["counts_sim"]
mean_diff = np.mean(np.abs(diff_percent))
print(f"Average relative difference: {mean_diff:.2f}%")

plt.figure(figsize=(20, 12))

plt.errorbar(histo["centers_data"], histo["counts_data"], yerr=histo["errors_data"], fmt='o', color='blue', label='LH2 Data(combined runs)')
plt.errorbar(histo["centers_sim"], histo["counts_sim"], yerr=histo["errors_sim"], fmt='s', color='black', label='SIMC')
plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='red', label="Dummy/random-subtracted data")
plt.xlabel("P.gtr.dp [%]", fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(0, max(histo["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title("SHMS delta")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

# HMS ytar

In [ ]:
plt.style.use("fivethirtyeight")
bin_num = 60
var_to_plot = ["H.gtr.y","hsytar"]
bin_range = (-3, 3)

histo = make_histograms(bin_num, bin_range)

diff_percent = 100 * (histo["counts_sub"] - histo["counts_sim"]) / histo["counts_sim"]
mean_diff = np.mean(np.abs(diff_percent))
print(f"Average relative difference: {mean_diff:.2f}%")

plt.figure(figsize=(20, 12))

plt.errorbar(histo["centers_data"], histo["counts_data"], yerr=histo["errors_data"], fmt='o', color='blue', label='LH2 Data(combined runs)')
plt.errorbar(histo["centers_sim"], histo["counts_sim"], yerr=histo["errors_sim"], fmt='s', color='black', label='SIMC')
plt.errorbar(histo["centers_dummy"], histo["counts_dummy"], yerr=histo["errors_dummy"], fmt='o', color='red', label="Dummy")
plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='green', label="Dummy/random-subtracted data")
plt.xlabel("H.gtr.y [cm]", fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(0, max(histo["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title("HMS ytar")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

# SHMS ytar

In [ ]:
plt.style.use("fivethirtyeight")
bin_num = 40
var_to_plot = ["P.gtr.y","ssytar"]
bin_range = (-2, 2)

histo = make_histograms(bin_num, bin_range)

diff_percent = 100 * (histo["counts_sub"] - histo["counts_sim"]) / histo["counts_sim"]
mean_diff = np.mean(np.abs(diff_percent))
print(f"Average relative difference: {mean_diff:.2f}%")

plt.figure(figsize=(20, 12))

plt.errorbar(histo["centers_data"], histo["counts_data"], yerr=histo["errors_data"], fmt='o', color='blue', label='LH2 Data (combined runs)')
plt.errorbar(histo["centers_sim"], histo["counts_sim"], yerr=histo["errors_sim"], fmt='s', color='black', label='SIMC')
plt.errorbar(histo["centers_dummy"], histo["counts_dummy"], yerr=histo["errors_dummy"], fmt='o', color='red', label="Dummy")
plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='green', label="Dummy/random-subtracted data")
plt.xlabel("P.gtr.y [cm]", fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(0, max(histo["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title("SHMS ytar")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

# Bjorken-x

In [ ]:
plt.style.use("fivethirtyeight")
from scipy.optimize import curve_fit

bin_num = 40
var_to_plot = ["H.kin.primary.x_bj","xbj"]
bin_range = (0.15, 0.35)

histo = make_histograms(bin_num, bin_range)

def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

mask = histo["counts_sub"] > 0
x_fit = histo["centers_sub"][mask]
y_fit = histo["counts_sub"][mask]
y_err = histo["errors_sub"][mask]

p0 = [np.max(y_fit), np.mean(x_fit), np.std(x_fit)]

popt, pcov = curve_fit(gaussian, x_fit, y_fit, p0=p0, sigma=y_err, absolute_sigma=True)

A_fit, mu_fit, sigma_fit = popt
A_err, mu_err, sigma_err = np.sqrt(np.diag(pcov))


mask = histo["counts_sim"]>0
diff_percent = 100 * (histo["counts_sub"][mask] - histo["counts_sim"][mask]) / histo["counts_sim"][mask]
mean_diff = np.mean(np.abs(diff_percent))
print(f"Average relative difference: {mean_diff:.2f}%")
print(f"μ = {mu_fit:.4f} ± {mu_err:.4f}")

plt.figure(figsize=(20, 12))

plt.errorbar(histo["centers_data"], histo["counts_data"], yerr=histo["errors_data"], fmt='o', color='blue', label='LH2 Data(combined runs)')
plt.errorbar(histo["centers_sim"], histo["counts_sim"], yerr=histo["errors_sim"], fmt='s', color='black', label='SIMC')
plt.errorbar(histo["centers_dummy"], histo["counts_dummy"], yerr=histo["errors_dummy"], fmt='o', color='red', label="Dummy")
plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='green', label="Dummy/random-subtracted data")
x_smooth = np.linspace(np.min(x_fit), np.max(x_fit), 500)
plt.plot(x_smooth, gaussian(x_smooth, *popt), color='green', linestyle="dotted", label="Gaussian fit")
plt.xlabel("Bjorken-x (momentum fraction)", fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(0, max(histo["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title("Bjorken-x")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

# Q^2

In [ ]:
plt.style.use("fivethirtyeight")
# plt.style.use("classic")
bin_num = 75
var_to_plot = ["H.kin.primary.Q2", "Q2"]
bin_range = (2.5, 4)

histo = make_histograms(bin_num, bin_range)

def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

mask = histo["counts_sub"] > 0
x_fit = histo["centers_sub"][mask]
y_fit = histo["counts_sub"][mask]
y_err = histo["errors_sub"][mask]

p0 = [np.max(y_fit), np.mean(x_fit), np.std(x_fit)]

popt, pcov = curve_fit(gaussian, x_fit, y_fit, p0=p0, sigma=y_err, absolute_sigma=True)

A_fit, mu_fit, sigma_fit = popt
A_err, mu_err, sigma_err = np.sqrt(np.diag(pcov))


diff_percent = 100 * (histo["counts_sub"][mask] - histo["counts_sim"][mask]) / histo["counts_sim"][mask]
mean_diff = np.mean(np.abs(diff_percent))
print(f"Average relative difference: {mean_diff:.2f}%")
print(f"μ = {mu_fit:.4f} ± {mu_err:.4f}")
plt.figure(figsize=(20, 12))

plt.errorbar(histo["centers_data"], histo["counts_data"], yerr=histo["errors_data"], fmt='o', color='blue', label='LH2 Data(combined runs)')
plt.errorbar(histo["centers_sim"], histo["counts_sim"], yerr=histo["errors_sim"], fmt='s', color='black', label='SIMC')
plt.errorbar(histo["centers_dummy"], histo["counts_dummy"], yerr=histo["errors_dummy"], fmt='o', color='red', label="Dummy")
plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='green', label="Dummy/random-subtracted data")
x_smooth = np.linspace(np.min(x_fit), np.max(x_fit), 500)
plt.plot(x_smooth, gaussian(x_smooth, *popt), color='green', linestyle="dotted", label="Gaussian fit")
plt.xlabel(r"$Q^2$ (GeV)", fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(0, max(histo["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title(r"$Q^2$")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

In [ ]:
plt.style.use("fivethirtyeight")
bin_num = 100
bin_range = (0.4,0.7)

var_to_plot = ["P.gtr.p", "z"]
histo_p = make_histograms(bin_num, bin_range)

var_to_plot = ["H.kin.primary.nu", "z"]
histo_nu = make_histograms(bin_num, bin_range)

mask = histo_nu["counts_data"] > 0
data_z = histo_p["counts_data"][mask] / histo_nu["counts_data"][mask]
errors_data = histo_p["errors_data"][mask]


plt.figure(figsize=(20, 12))

plt.errorbar(histo_p["centers_data"][mask], data_z, yerr=errors_data, fmt='o', color='blue', label='LH2 Data (combined runs)')
plt.errorbar(histo_p["centers_sim"], histo_p["counts_sim"], yerr=histo_p["errors_sim"], fmt='s', color='black', label='SIMC')
# plt.errorbar(histo["centers_dummy"], histo["counts_dummy"], yerr=histo["errors_dummy"], fmt='o', color='red', label="Dummy")
# plt.errorbar(histo["centers_sub"], histo["counts_sub"], yerr=histo["errors_sub"], fmt='o', color='green', label="Dummy/random-subtracted data")
plt.xlabel("z", fontsize=25)
plt.ylabel("Normalized yield", fontsize=25)
plt.ylim(0, max(histo_p["counts_data"])*1.2)
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.minorticks_on()
ax=plt.gca()
ax.set_frame_on(True)
for spine in ax.spines.values():
    spine.set_linewidth(3)
    spine.set_color('black')
plt.tick_params(which='major', length=15, width=3, direction='in', bottom=True, top=True, left=True, right=True)
plt.tick_params(which='minor', length=8, width=1, direction='in', bottom=True, top=True, left=True, right=True)
plt.rcParams["axes.titlesize"] = 40  
plt.title("z")
plt.legend(fontsize=13)
plt.grid(True)
plt.show()

In [ ]:
histo_p["counts_data"]

In [ ]:
z=0.5, pass 4, pi+